In [136]:
import pandas as pd
import numpy as np
from math import log

def create_data():
    df = pd.read_excel(r"银行借贷数据集train.xls")
    # revenue离散化
    re = [0,10000, 20000,30000,40000,50000]
    df['revenue'] = pd.cut(df['revenue'],re,labels=False)
    # 获取数据集和每个维度的名称
    df = df.drop(['nameid'], axis=1)  # 假设有ID列需要移除
    datasets = df.values
    labels = df.columns.values
    return datasets, labels

datasets, labels = create_data()
train_data = pd.DataFrame(datasets, columns=labels)

test_df = pd.read_excel(r"银行借贷数据集test.xls")
re = [0,10000, 20000,30000,40000,50000]
test_df["revenue"] = pd.cut(test_df["revenue"],re,labels=False)
X_test = test_df.drop(['nameid', 'approve'], axis=1)  # 假设最后一列是目标变量
y_test = test_df['approve']


In [137]:
class Node:
    def __init__(self, root=True, label=None, feature_name=None, feature=None):
        self.root = root
        self.label = label
        self.feature_name = feature_name
        self.feature = feature
        self.tree = {}
        self.result = {'label:': self.label, 'feature': self.feature, 'tree': self.tree}

    def __repr__(self):
        return '{}'.format(self.result)

    def add_node(self, val, node):
        self.tree[val] = node

    def predict(self, features):
        if self.root is True:
            return self.label
        if features[self.feature] not in self.tree:
            key_feature = list(self.tree.keys())  # 储存特征出现过的取值
            x = np.random.randint(0, len(key_feature))  # 随机生成一个数字
            random_key = key_feature[x]
            return self.tree[random_key].predict(features)
        return self.tree[features[self.feature]].predict(features)

In [138]:
class DTree:
    def __init__(self, epsilon=0.017):
        self.epsilon = epsilon
        self._tree = {}

    # 熵
    def calc_ent(self, datasets):
        data_length = len(datasets)
        label_count = {}
        for i in range(data_length):
            label = datasets[i][-1]
            if label not in label_count:
                label_count[label] = 0
            label_count[label] += 1
        ent = -sum([(p/data_length)*log(p/data_length, 2) for p in label_count.values()])
        return ent

    # 经验条件熵
    def cond_ent(self, datasets, axis=0):
        data_length = len(datasets)
        feature_sets = {}
        for i in range(data_length):
            feature = datasets[i][axis]
            if feature not in feature_sets:
                feature_sets[feature] = []
            feature_sets[feature].append(datasets[i])
        cond_ent = sum([(len(p)/data_length)*self.calc_ent(p) for p in feature_sets.values()])
        return cond_ent

    # 信息增益比
    def info_gain(self, ent, cond_ent):
        if cond_ent == 0:
            return 100000
        else:
            return (ent - cond_ent) / cond_ent

    # 返回信息增益比最大的特征
    def info_gain_train(self, datasets):
        count = len(datasets[0]) - 1
        ent = self.calc_ent(datasets)
        best_feature = []
        for c in range(count):
            c_info_gain = self.info_gain(ent, self.cond_ent(datasets, axis=c))
            best_feature.append((c, c_info_gain))
        # 比较大小
        best_ = max(best_feature, key=lambda x: x[-1])
        return best_

    def train(self, train_data):
        _, y_train, features = train_data.iloc[:, :-1], train_data.iloc[:, -1], train_data.columns[:-1]

        if len(y_train.value_counts()) == 1:
            return Node(root=True, label=y_train.iloc[0])

        if len(features) == 0:
            return Node(root=True, label=y_train.value_counts().sort_values(ascending=False).index[0])

        max_feature, max_info_gain = self.info_gain_train(np.array(train_data))
        max_feature_name = features[max_feature]

        if max_info_gain < self.epsilon:
            return Node(root=True, label=y_train.value_counts().sort_values(ascending=False).index[0])

        node_tree = Node(root=False, feature_name=max_feature_name, feature=max_feature)

        feature_list = train_data[max_feature_name].value_counts().index
        for f in feature_list:
            sub_train_df = train_data.loc[train_data[max_feature_name] == f].drop([max_feature_name], axis=1)

            sub_tree = self.train(sub_train_df)
            node_tree.add_node(f, sub_tree)
        
        return node_tree

    def fit(self, train_data):
        self._tree = self.train(train_data)
        return self._tree

    def predict(self, X_test):
        return self._tree.predict(X_test)


In [139]:
# 创建并训练决策树
dt = DTree()
tree = dt.fit(train_data)
print(tree)

# 使用模型进行预测
predictions = X_test.apply(lambda x: dt.predict(x), axis=1)

# 计算评估指标
def calculate_precision_recall_f1(y_true, y_pred):
    tp = sum((y_true == 1) & (y_pred == 1))
    fp = sum((y_true == 0) & (y_pred == 1))
    fn = sum((y_true == 1) & (y_pred == 0))
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    return precision, recall, f1

# 评估模型性能
precision, recall, f1 = calculate_precision_recall_f1(y_test, predictions)
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


{'label:': None, 'feature': 2, 'tree': {1: {'label:': None, 'feature': 2, 'tree': {0: {'label:': None, 'feature': 4, 'tree': {1: {'label:': None, 'feature': 1, 'tree': {4: {'label:': None, 'feature': 0, 'tree': {4: {'label:': None, 'feature': 1, 'tree': {1: {'label:': 1, 'feature': None, 'tree': {}}, 0: {'label:': None, 'feature': 0, 'tree': {0: {'label:': 0, 'feature': None, 'tree': {}}, 1: {'label:': 1, 'feature': None, 'tree': {}}}}}}, 2: {'label:': 1, 'feature': None, 'tree': {}}, 3: {'label:': 1, 'feature': None, 'tree': {}}, 5: {'label:': None, 'feature': 1, 'tree': {1: {'label:': 0, 'feature': None, 'tree': {}}, 0: {'label:': 1, 'feature': None, 'tree': {}}}}, 1: {'label:': 1, 'feature': None, 'tree': {}}}}, 3: {'label:': None, 'feature': 2, 'tree': {1: {'label:': None, 'feature': 0, 'tree': {3: {'label:': 1, 'feature': None, 'tree': {}}, 1: {'label:': None, 'feature': 0, 'tree': {0: {'label:': 0, 'feature': None, 'tree': {}}, 1: {'label:': 1, 'feature': None, 'tree': {}}}}, 5: 